In [7]:
# -*- coding: utf-8 -*-

import requests
from bs4 import BeautifulSoup as soup
import csv
import xlsxwriter


workbook = xlsxwriter.Workbook('otodom_results.xlsx')
worksheet = workbook.add_worksheet()

row = 0
col = 0

worksheet.write(row, col, "nazwa")
worksheet.write(row, col + 1, "powierzchnia w m²")
worksheet.write(row, col + 2, "cena w zł")
worksheet.write(row, col + 3, "cena za metr")
worksheet.write(row, col + 4, "adres")

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"}
results = "https://www.otodom.pl/wynajem/lokal/warszawa/?page={}"

with requests.session() as s:
    req = s.get(results.format(1), headers=headers)
    page_soup = soup(req.text, "html.parser")
    total_pages = int(page_soup.find("div",{"class":"after-offers clearfix"}).find("ul",{"class":"pager"}).findAll("li")[4].text)
    print('ZNALEZIONO STRON: ', total_pages)
    cnt = 1
    for page in range(1, total_pages):
        req = s.get(results.format(page), headers=headers)
        page_soup = soup(req.text, "html.parser")
        output_string = ""
        collection = page_soup.select('div.offer-item-details')
    
        for a in collection:
            try:
                offer_name = a.find('span', attrs={'class': 'offer-item-title'}).text
                area = a.find('strong', attrs={'class' : 'visible-xs-block'}).text.replace(" m²", "")
                price = a.find('li', attrs={'class', 'offer-item-price'}).text.replace("\n", "").replace(" ", "").replace("zł/mc", "")
                price_per_meter = a.find('li', attrs={'class', 'offer-item-price-per-m'}).text.replace(" zł/m²", "")
                card_url = a.find('h3').find('a').get('href')
                card_page = s.get(card_url)
                card_data = soup(card_page.content, 'html.parser')
                location = card_data.find('a', attrs={'class', 'css-1rg0v62'}).text.replace('\"', "")
            except:
                pass
            output_string = offer_name + "|" + area + "|" + price + "|" + price_per_meter + "|" + location + "|"
            row += 1
            worksheet.write(row, col, offer_name)
            worksheet.write(row, col + 1, area)
            worksheet.write(row, col + 2, price)
            worksheet.write(row, col + 3, price_per_meter)
            worksheet.write(row, col + 4, location)
            

            print(cnt, output_string)
            cnt += 1
workbook.close()

ZNALEZIONO STRON:  188
1 490 m2 | 17 pokoi | biuro w samym centrum Warszawy|486|35000|72|Warszawa, Śródmieście, Plac Konstytucji|
2 Lokal parterowy z witryną| 137m2| pl Trzech Krzyży|137|48000|350|Warszawa, Śródmieście, Śródmieście Południowe, Plac Trzech Krzyży|
3 Wynajmę biuro 3 pokojowe 81,5 m2 ul. Madalińskiego|81,60|3700|45|Warszawa, Mokotów, ul. Antoniego Józefa Madalińskiego|
4 Do wynajęcia lokal w samym centrum Warszawy|58|5800|100|Warszawa, Śródmieście, Foksal|
5 Lokal użytkowy, 90 m2 + 30 m2 TARAS, Mokotów|88,40|6950|79|Warszawa, Mokotów, Górny Mokotów|
6 Wola ul. Giełdowa ,biuro, Super Standard|12|800|67|Warszawa, Wola, Giełdowa|
7 Lokal Praga Południe 33 m2|33|2800|85|Warszawa, Praga-Południe, ul. Wojciecha Chrzanowskiego|
8 Budynek biurowy 1770m2, klasa A, open-space, Wola|1 856,38|138591|75|Warszawa, Wola|
9 Budynek biurowy 1062m2, klasa A, open-space, Wola|1 113,83|83154,60|75|Warszawa, Wola|
10 Lokal biurowy 209m2, klasa A, open-space, Wola|219,20|16364,70|75|Warszawa, 

84 Krótka umowa przy metrze, budynek klasy A|284|~22174|78|Warszawa, Wola, Mirów|
85 Wiertnicza | reprezentacyjny i nowoczesny dom|240|15000|63|Warszawa, Wilanów, Wiertnicza|
86 Śródmieście | 9 pomieszczeń | przedwojenny | ogród|300|13500|45|Warszawa, Śródmieście, Śródmieście Południowe, Lekarska|
87 Saska Kępa | Willa na cele biurowe | 6 pokoi ||220|12000|55|Warszawa, Praga-Południe, Saska Kępa, Zakopiańska|
88 Lokal Gabinety Biuro 145/290m okolice ulMarsa BEZP|145|5900|41|Warszawa, Praga-Południe|
89 Bezpośrednio, od ulicy ,duze witryny, 34m2|34|2100|62|Warszawa, Białołęka, Świderska 113|
90 Lokal usługowy Stara Miłosna|47|1800|38|Warszawa, Wesoła, Stara Miłosna|
91 lokal na ul. KEN ,Kabaty, Atrakcyjna lokalizacja,|87|8300|95|Warszawa, Ursynów, Kabaty, KEN 24|
92 Do wynajecia lokal biurowy|110|4700|43|Warszawa, Ochota|
93 Oddzielny budynek pod usługi, produkcje, magazyn|40|1700|43|Warszawa, Ursus|
94 Bezposrednio 10 Lok. użyt. od 20 do 500 m|100|3800|38|Warszawa, Włochy, Al. Krakowsk

173 BEZPOŚREDNIO NOWY lokal BIUROWY 100M2 funkcjonalny|100|4500|45|Warszawa, Ursynów, ul. Żołny|
174 Lokal biurowy do wynajęcia, Łodygowa 3, Warszawa|24|2000|83|Warszawa, Targówek, Targówek Mieszkaniowy, ul. Łodygowa 3|
175 Biuro - Klinika, Ochota, Włodarzewska|250|~12661|51|Warszawa, Ochota, ul. Włodarzewska|
176 Bezpośrednio lokal usługowy 72m2, URSYNÓW, Natolin|71,75|3100|43|Warszawa, Ursynów, Lanciego|
177 Lokal do wynajęcia w ŚCISŁYM CENTRUM|26|12000|462|Warszawa, Śródmieście|
178 Wynajme lokal użytkowy Warszawa Nowolipki 13|58|5600|97|Warszawa, Śródmieście, Nowolipki 13|
179 Lokal usługowy Mokotów Mangalia|42,86|2800|65|Warszawa, Mokotów, Dolny Mokotów, ul. Mangalia|
180 Kameralny lokal Rembertów do wynajęcia|34|2040|60|Warszawa, Rembertów|
181 Lokal 115m2 Rembertów do wynajęcia|115|6900|60|Warszawa, Rembertów|
182 Lokal do wynajęcia na ulicy Karolkowej|89|6400|72|Warszawa, Wola, ul. Jaktorowska|
183 Budynek biurowy 2333m2, klasa A, open-space, Wola|2 446,88|182673,90|75|Warszawa

261 Lokal z witrynami na zielonym Żoliborzu|80,50|9800|122|Warszawa, Żoliborz, Sady Żoliborskie, Przasnyska|
262 Mokotowska róg Chopina-lokal do wynajęcia 108,3m2|108,30|24990|231|Warszawa, Śródmieście, ul. Mokotowska|
263 137m2 - Przestrzeń biurowa Praga Północ|137|12000|88|Warszawa, Praga-Północ, ul. Kijowska|
264 Lokal handlowo-gastronomiczny do wynajmu|127,69|10900|85|Warszawa, Ursus, Bohaterów Warszawy 26|
265 95m2 - lokal usługowy Kasprowicza - METRO WAWRZYSZ|95|10500|111|Warszawa, Bielany, ul. Jana Kasprowicza|
266 Okazja Biurowiec Żoliborz|1 029|59394|58|Warszawa, Żoliborz|
267 Biuro 242 m2 Mokotów|242|15000|62|Warszawa, Mokotów, ul. Belwederska 9A|
268 65m2 ul. Cybernetyki 15 BEZPOŚREDNIO|65|6440|99|Warszawa, Mokotów, Służew, ul. Cybernetyki|
269 Idealny lokal na magazyn blisko metra M2 na Pradze|200|10000|50|Warszawa, Praga-Północ|
270 Lokal gastronomiczny Al. Rzeczypospolitej 6|114|9120|80|Warszawa, Wilanów, al. Rzeczypospolitej|
271 Róg Rydygiera/Przasnyska. Wykończony. B. 

346 Rondo Daszyńskiego duży lokal|163|14670|90|Warszawa, Wola|
347 GOTOWY LOKAL WYNAJMĘ - ul. Twarda, Śródmieście|25|1830|73|Warszawa, Śródmieście|
348 Lokal Usługowy 29m2,ul. Herbu Oksza Warszawa Ursus|29|2500|86|Warszawa, Ursus, Herbu Oksza|
349 Lokal użytkowy, 301 m², Warszawa|301|~25406|84|Warszawa, Śródmieście|
350 Lokal użytkowy, 566 m², Warszawa|566|~47774|84|Warszawa, Śródmieście|
351 Lokal użytkowy, 791 m², Warszawa|791|~66765|84|Warszawa, Śródmieście|
352 Saska Kępa | 260m2 | Biuro z tarasem na dachu|260|11000|42|Warszawa, Praga-Południe, Saska Kępa|
353 Lokal Biurowy na Żoliborzu, Bezpośrednio|30|1990|66|Warszawa, Żoliborz|
354 Lokal usługowy na wynajem - Sadyba - 84m2|84,40|7500|89|Warszawa, Mokotów, Sadyba, Bonifacego 92|
355 Lokal użytkowy, 465 m², Warszawa|465|~39249|84|Warszawa, Śródmieście|
356 Lokal 107 m2 kamienica, centrum, Al. Jerozolimskie|106,82|4900|46|Warszawa, Śródmieście, Aleje Jerozolimskie|
357 Lokal/Biuro na przeciwko Auchan Okęcie.Super Cena|140|5000|36|W

...

4625 Biuro Na Mokotowie blisko metra !!!|708|~43326|61|Warszawa, Mokotów|
4626 Lokal użytkowy, 331,75 m², Warszawa|331,75|23223|70|Warszawa, Wola, Mirów, Żelazna|
4627 Lokal na wynajem w prestiżowym miejscu|110|~7428|68|Warszawa, Włochy|
4628 Lokal na wynajem|475|38000|80|Warszawa, Targówek|
4629 Lokal przy wyjściu z metra|160|19200|120|Warszawa, Bielany, Młociny|
4630 Lokal użytkowy, 1 300 m², Warszawa|1 300|32500|25|Warszawa, Śródmieście|
4631 Lokal użytkowy, 120 m², Warszawa|120|~10635|89|Warszawa, Śródmieście|
4632 Ekologiczny i wielofunkcyjny kompleks biurowy|450|~24689|55|Warszawa, Włochy|
4633 Lokal handlowy na Dolnym Mokotowie|251|~16949|68|Warszawa, Mokotów|
4634 Projekt biurowo-usługowy klasy A na bliskiej Woli|3 813|~354025|93|Warszawa, Wola|
4635 Kameralny budynek biurowy na Mokotowie|171|8550|50|Warszawa, Mokotów|
4636 Biuro przy Rondzie Radosława|150|~18991|127|Warszawa, Śródmieście|
4637 Kamienica biurowa w ścisłym centrum Warszawy|188|~19835|106|Warszawa, Śródmieście|
4

4711 Biurko dedykowane - Park Avenue|10|870|87|Warszawa, Wesoła, ul. Wspólna 70|
4712 Członkostwo Platinum - Park Avenue|5|529|106|Warszawa, Wesoła, ul. Wspólna 70|
4713 Salon Biznesowy - Park Avenue|5|239|48|Warszawa, Wesoła, ul. Wspólna 70|
4714 Biura typu open plan - Skylight|45|11685|260|Warszawa, Śródmieście, ul. Złota 59|
4715 Biura 5 przestrzeni do pracy - Skylight|30|7345|245|Warszawa, Śródmieście, ul. Złota 59|
4716 Biura 3-4 przestrzeni do pracy - Skylight|15|3670|245|Warszawa, Śródmieście, ul. Złota 59|
4717 Biura 1-2 przestrzeni do pracy - Skylight|10|2447|245|Warszawa, Śródmieście, ul. Złota 59|
4718 Biurko dedykowane - Skylight|10|950|95|Warszawa, Śródmieście, ul. Złota 59|
4719 Członkostwo Platinum - Skylight|5|529|106|Warszawa, Śródmieście, ul. Złota 59|
4720 Salon Biznesowy - Skylight|5|239|48|Warszawa, Śródmieście, ul. Złota 59|
4721 Biura typu open plan - North Gate|45|10470|233|Warszawa, Śródmieście, ul. Bonifraterska 17|
4722 Biura 5 przestrzeni do pracy - North Ga

4804 Jerozolimskie 200 Najlepsze Biuro w Mieście (Loft)|21|1490|71|Warszawa, Włochy, al. Jerozolimskie|
4805 Biuro Wilanów , Kancelaria, Biuro Rachunkowe|167,75|8387,50|50|Warszawa, Wilanów|
4806 Biuro na Woli blisko metra !!!|11|660|60|Warszawa, Wola|
4807 BIURO ok. 70 metrów|70|2900|41|Warszawa, Praga-Południe, Ostrobramska  |
4808 80 m2 -Przestrzeń biurowa Praga Północ|80|6500|81|Warszawa, Praga-Północ, ul. Kijowska|
4809 Klimatyczne biuro na Mokowskiej!|151|8500|56|Warszawa, mazowieckie|
4810 Biuro na Woli blisko Metra !!!|16,80|940,80|56|Warszawa, Wola|
4811 Lokal użytkowy na Woli przy Metrze|89|6000|67|Warszawa, Wola|
4812 Saska Kępa ekskluzywne 71 m2|71|4500|63|Warszawa, Praga-Południe, Saska Kępa|
4813 Lokal Usługowy Na Dużym Osiedlu|82|5000|61|Warszawa, Ursynów, ul. Kłobucka|
4814 Lokal Usługowy Na Dużym Osiedlu|104|6000|58|Warszawa, Ursynów, ul. Kłobucka|
4815 Śródmieście, 584 m2, Parter, Witryna|584|55480|95|Warszawa, Śródmieście, al. Jana Pawła II|
4816 Wola lokal han-usł. 

4901 Biuro 3 pokojowe w centrum Warszawy|71,92|4099,44|57|Warszawa, Śródmieście|
4902 Biuro 2 pokojowe w centrum Warszawy|45|2475|55|Warszawa, Śródmieście|
4903 Lokal Warszawa Grójecka/Spiska|168|9900|59|Grójecka/Spiska, Warszawa, mazowieckie|
4904 lokal usługowy ścisłe centrum Oboźna|62|5500|89|Warszawa, Śródmieście|
4905 Biuro 582 m2 na Ursynowie !|582|~27018|46|Warszawa, Ursynów, Wyczółki|
4906 Biuro 270 m2 na Ursynowie !|270|~12534|46|Warszawa, Ursynów, Wyczółki|
4907 Powierzchnia biurowa na Ursynowie.|309|~14349|46|Warszawa, Ursynów, Wyczółki|
4908 Reprezentacyjne biuro- 582 m2 !|582|~27010|46|Warszawa, Ursynów, Wyczółki|
4909 Reprezentacyjne biuro- 753 m2 !|753|~34957|46|Warszawa, Ursynów, Wyczółki|
4910 Biuro na Ursynowie- 376 m2 !|376|~17455|46|Warszawa, Ursynów, Wyczółki|
4911 Mały moduł biurowy- 150 m2 !|148|~6871|46|Warszawa, Ursynów, Wyczółki|
4912 Biura do wynajęcia blisko metra Świętokrzyska|244|~17506|72|Warszawa, Śródmieście|
4913 Nowe biuro, ul.Żegańska, Wawer, Międzyl

5003 Atrakcyjna powierzchnia biurowa, okolice Lotniska|1 200|42000|35|Warszawa, Włochy, Okęcie|
5004 Luksusowy lokal w kamienicy blisko Nowego Swiatu|104|8700|84|Warszawa, mazowieckie|
5005 Lokal użytkowy, 80 m², Warszawa|80|6800|85|Warszawa, Śródmieście|
5006 Do wynajęcia lokal na alejach Jerozolimskich|2 500|105000|42|Warszawa, Włochy|
5007 Powierzchnia do wynajęcia na al. Jerozolimskich|5 000|210000|42|Warszawa, Włochy|
5008 Lokal użytkowy, 20 m², Warszawa|20|840|42|Warszawa, Włochy|
5009 Lokal do wynajęcia na Nowym Świecie|40|3400|85|Warszawa, Śródmieście|
5010 Powierzchnia do wynajęcia w samym centrum|60|5100|85|Warszawa, Śródmieście|
5011 Powierzchnie biurowe do wynajęcia w samym centrum!|20|1700|85|Warszawa, Śródmieście|
5012 Galeria Meblowa i nie tylko Al.Jerozolimskie|108|4860|45|Warszawa, Włochy, Okęcie|
5013 Jerozolimskie od 45-1000 m2 handlu bezposrednio|1 000|~2954|3|Warszawa, Włochy, Salomea|
5014 Nowy Świat lokal wielofunkcyjny Bezpośrednio|680|~54526|80|Warszawa, Śródmi